# Jonathan Halverson
# Monday, August 7, 2017
# Chapter 6: Statistical machine learning

Structure-based for data-driven methods: Linear and logistic regression are said to be structure-based while KNN and tree methods are data-driven.